# Motivation
Natural language texts often have a very high frequency of certain letters, in German for example, almost every 5th letter is an E, but only every 500th is a Q. It would then be clever to choose a very small representation for E. This is exactly what the Huffman compression is about, choosing the length of the representation based on the frequencies of the symbol in the text.

# Algorithm
Let's assume we want to encode the word "aaaabcc", then we calculate the frequencies of the letters in the text:
Symbol 	Frequency
a 	4
b 	1
c 	2

Now we choose a smaller representation the more often it occurs, to minimize the overall space needed. The algorithm uses a tree for encoding and decoding:

  .
 / \
a   .
   / \
   b  c

Usually we choose 0 for the left branch and 1 for the right branch (but it might also be swapped). By traversing from the root to the symbol leaf, we want to encode, we get the matching representation. To decode a sequence of binary digits into a symbol, we start from the root and just follow the path in the same way, until we reach a symbol.

Considering the above tree, we would encode a with 0, b with 10 and c with 11. Therefore encoding "aaaabcc" will result in 0000101111.

(Note: As you can see the encoding is not optimal, since the code for b and c have same length, but that is topic of another data compression Kata.)
# Tree construction

To build the tree, we turn each symbol into a leaf and sort them by their frequency. In every step, we remove 2 trees with the smallest frequency and put them under a node. This node gets reinserted and has the sum of the frequencies of both trees as new frequency. We are finished, when there is only 1 tree left.

(Hint: Maybe you can do it without sorting in every step?)
# Goal

Write functions frequencies, encode and decode.

Note: Frequency lists with just one or less elements should get rejected. (Because then there is no information we could encode, but the length.)

# Notes from Test Cases

1. the higher frequencies are, the lower the length of their encoding should be (can also be equal for non-powers of 2)
2. for an alphabet with same frequencies and with size of 2^n all encodings should have the length n

In [187]:
# takes: str; returns: [ (str, int) ] (Strings in return value are single characters)
def frequencies(s):
    from collections import Counter
    frequencies = Counter(s)
    new_list = sorted(s, key=lambda x: (frequencies[x], x), reverse=False)
    letters = list(dict.fromkeys(new_list))
    if len(letters) == 0:
        return []
    freqs_list = []
    for char in letters:
         freqs_list.append((char, s.count(char)))
    return freqs_list

In [213]:
def combine_freq(freqs):   
    # find the entry with the lowest frequency
    freq_num = [freq for (char, freq) in freqs]
    pos1 = freq_num.index(min(freq_num))

    # the entry with the lowest frequency is not necessarily the first entry
    # to find the entry with the sescond lowest frequency, first split the frequency table into two
    low_before = low_after = max(freq_num) + 1
    if pos1 > 0:
         low_before = min(freq_num[0:pos1])
    if pos1 < len(freqs)-1:
        low_after = min(freq_num[pos1+1:])
    pos2 = freq_num.index(min(low_before, low_after))
    if pos2 == pos1:
        pos2 = freq_num.index(min(low_before, low_after), pos1+1)

    # combine the 2 entries with the lowest frequencies to create a new entry in the list
    if pos2 < pos1:
        pos1, pos2 = pos2, pos1
    new_entry = [((freqs[pos1][0], freqs[pos2][0]), min(freq_num) + min(low_before, low_after))]
    freqs_adj = new_entry + freqs[0:pos1] + freqs[pos1+1:pos2] + freqs[pos2+1:]

    return freqs_adj

In [194]:
def get_branch(dict, branch, digit, letter_first):
    if len(branch) == 1:
        if letter_first:
            dict.update({branch : digit})
        else:
            dict.update({digit : branch})
    else:
        dict = get_branch( dict, branch[0], digit + '0', letter_first )
        dict = get_branch( dict, branch[1], digit + '1', letter_first )
    return dict

In [159]:
def create_tree(freqs, letter_first):
    freqs_adj = freqs[:]
    while len(freqs_adj) > 2:
        freqs_adj = combine_freq(freqs_adj)

    # should now have 2 entries
    # the first entry will be the '0' branch of the tree
    dict = {}
    dict = get_branch( dict, freqs_adj[0][0], '0', letter_first )

    # the second entry will be the '1' branch of the tree
    dict = get_branch( dict, freqs_adj[1][0], '1', letter_first )
    return dict

In [169]:
def single_level(freqs, power, letter_first):
    dict = {}
    for i in range(2**power):
        # convert the value to its equivalent binary value (ignoring the 0b at the start)
        bin_value = str(bin(i))[2:]

        # pad the value with zeroes to the number of digits given by the number of levels 
        entry = '0' * (power - len(bin_value)) + bin_value

        # append entry to dictionary
        if letter_first:
            dict.update({freqs[i][0] : entry})
        else:
            dict.update({ entry: freqs[i][0]})
    return dict

In [183]:
def create_dict(freqs, letter_first):
    # determine whether the length of the frequency table is a power of 2
    power = 1
    while 2**power < len(freqs):
        power += 1

    # if the length of the frequency table is an exact power of 2, and all entries have the same frequency, create the tree all on one level
    freq_num = [freq for (char, freq) in freqs]
    if 2**power == len(freqs) and min(freq_num) == max(freq_num):
        dict = single_level(freqs, power, letter_first)

    # not a power of 2, so need to create tree, with high-frequency entries higher up the tree than low-frequency entries
    else:
        dict = create_tree(freqs, letter_first)

    return dict

In [171]:
# takes: [ (str, int) ], str; returns: String (with "0" and "1")
def encode(freqs, s):
    if len(freqs) < 2:
        return None

    dict = create_dict(freqs, True)
    encoded = ''
    for char in s:
        if char in dict:
            encoded += dict[char]
    return encoded

In [172]:
# takes [ [str, int] ], str (with "0" and "1"); returns: str
def decode(freqs,bits):
    if len(freqs) < 2:
         return None

    dict = create_dict(freqs, False)
    bit = ''
    decoded = ''
    for char in bits:
        bit += char
        if bit in dict:
            decoded += dict[bit]
            bit = ''
    return decoded

In [214]:
fs = [('e', 2), ('f', 2), ('l', 4), ('m', 8), ('o', 16), ('q', 32), ('v', 64), ('z', 128)]
word = 'o'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'e': '0000000', 'f': '0000001', 'l': '000001', 'm': '00001', 'o': '0001', 'q': '001', 'v': '01', 'z': '1'}
encoded: 0001
decoded: o
decodes correctly: True


In [215]:
fs = [('e', 2), ('f', 2), ('l', 4), ('m', 8), ('o', 16), ('q', 32), ('v', 64), ('z', 128)]
word = 'q'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'e': '0000000', 'f': '0000001', 'l': '000001', 'm': '00001', 'o': '0001', 'q': '001', 'v': '01', 'z': '1'}
encoded: 001
decoded: q
decodes correctly: True


In [202]:
fs = [('b', 2), ('c', 2), ('d', 4), ('k', 8), ('l', 16), ('p', 32), ('r', 64), ('s', 128), ('t', 256), ('u', 512), ('v', 1024), ('x', 2048), ('y', 4096), ('z', 8192)]
print(len(fs))
word = 'c'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

14
{'b': '0000000000000', 'c': '0000000000001', 'd': '000000000001', 'k': '00000000001', 'l': '0000000001', 'p': '000000001', 'r': '00000001', 's': '0000001', 't': '000001', 'u': '00001', 'v': '0001', 'x': '001', 'y': '01', 'z': '1'}
encoded: 0000000000001
decoded: c
decodes correctly: True


In [203]:
fs = [('b', 1), ('c', 2), ('a', 4)]
word = 'aaaabcc'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'b': '00', 'c': '01', 'a': '1'}
encoded: 1111000101
decoded: aaaabcc
decodes correctly: True


In [204]:
fs = [('a', 1), ('b', 1)]
word = 'a'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'a': '0', 'b': '1'}
encoded: 0
decoded: a
decodes correctly: True


In [205]:
fs = [('a', 1), ('b', 2), ('c', 2), ('d', 3), ('e', 3), ('f', 3)]
word = 'abbccdddeeefff'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'a': '0000', 'b': '0001', 'c': '001', 'f': '01', 'd': '10', 'e': '11'}
encoded: 000000010001001001101010111111010101
decoded: abbccdddeeefff
decodes correctly: True


In [206]:
fs = [('b', 1), ('c', 2), ('a', 4)]
word = 'aaaabcc'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'b': '00', 'c': '01', 'a': '1'}
encoded: 1111000101
decoded: aaaabcc
decodes correctly: True


In [207]:
fs = [('k', 5), ('q', 5), ('o', 5), ('U', 5), ('W', 5), ('v', 5), ('!', 5), ('m', 5), ('?', 5), ('5', 5), ('T', 5), ('R', 5), ('e', 5), ('V', 5), ('B', 5), ('F', 5), ('D', 5), ('i', 5), ('X', 5), ('l', 5), ('n', 5), ('I', 5), ('1', 5), ('Y', 5), ('7', 5), ('u', 5), ('0', 5), ('Z', 5), ('P', 5), ('J', 5), ('M', 5), ('C', 5), ('4', 5), ('j', 5), ('g', 5), ('w', 5), ('x', 5), ('K', 5), ('A', 5), ('s', 5), ('9', 5), ('f', 5), ('r', 5), ('3', 5), ('Q', 5), ('b', 5), ('2', 5), ('G', 5), ('h', 5), ('d', 5), ('c', 5), ('N', 5), ('S', 5), ('O', 5), ('6', 5), ('t', 5), ('E', 5), ('H', 5), ('z', 5), ('y', 5), ('L', 5), ('p', 5), ('8', 5), ('a', 5)]
word = 'q'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'k': '000000', 'q': '000001', 'o': '000010', 'U': '000011', 'W': '000100', 'v': '000101', '!': '000110', 'm': '000111', '?': '001000', '5': '001001', 'T': '001010', 'R': '001011', 'e': '001100', 'V': '001101', 'B': '001110', 'F': '001111', 'D': '010000', 'i': '010001', 'X': '010010', 'l': '010011', 'n': '010100', 'I': '010101', '1': '010110', 'Y': '010111', '7': '011000', 'u': '011001', '0': '011010', 'Z': '011011', 'P': '011100', 'J': '011101', 'M': '011110', 'C': '011111', '4': '100000', 'j': '100001', 'g': '100010', 'w': '100011', 'x': '100100', 'K': '100101', 'A': '100110', 's': '100111', '9': '101000', 'f': '101001', 'r': '101010', '3': '101011', 'Q': '101100', 'b': '101101', '2': '101110', 'G': '101111', 'h': '110000', 'd': '110001', 'c': '110010', 'N': '110011', 'S': '110100', 'O': '110101', '6': '110110', 't': '110111', 'E': '111000', 'H': '111001', 'z': '111010', 'y': '111011', 'L': '111100', 'p': '111101', '8': '111110', 'a': '111111'}
encoded: 000001
decoded: q
decodes corr

In [208]:
fs = [('c', 1), ('d', 1), ('h', 1), ('i', 1), ('n', 1), ('s', 1), ('t', 1), ('v', 1), ('w', 1), ('y', 1), ('e', 2), ('g', 2), ('l', 2), ('m', 2), ('r', 2), ('a', 3), ('j', 3), ('o', 3), ('k', 4), ('q', 4), ('b', 5), ('z', 5)]
word = 'bkbabzazojqyjndkmzbrjqclhbreqelmkggvawokoziqtsz'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'k': '0000', 'q': '0001', 'n': '001000', 's': '001001', 'h': '001010', 'i': '001011', 'w': '001100', 'y': '001101', 't': '001110', 'v': '001111', 'm': '01000', 'r': '01001', 'o': '0101', 'a': '0110', 'j': '0111', 'b': '100', 'z': '101', 'g': '1100', 'l': '1101', 'c': '11100', 'd': '11101', 'e': '1111'}
encoded: 10000001000110100101011010101010111000100110101110010001110100000100010110001001011100011110011010010101000100111110001111111010100000001100110000111101100011000101000001011010010110001001110001001101
decoded: bkbabzazojqyjndkmzbrjqclhbreqelmkggvawokoziqtsz
decodes correctly: True


In [209]:
fs = [('d', 1), ('e', 1), ('f', 1), ('g', 1), ('h', 1), ('l', 1), ('r', 1), ('k', 2), ('p', 2), ('q', 2)]
word = 'repqdqpflhkgk'
print(create_dict(fs, True))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

{'k': '000', 'p': '001', 'f': '0100', 'g': '0101', 'd': '0110', 'e': '0111', 'h': '1000', 'l': '1001', 'r': '101', 'q': '11'}
encoded: 1010111001110110110010100100110000000101000
decoded: repqdqpflhkgk
decodes correctly: True


In [210]:
word = 'aaaabcc'
fs = frequencies(word)
print('fs', fs)
print('dict', create_dict(fs, True))
print('dict', create_dict(fs, False))
print('encoded:', encode(fs, word))
print('decoded:', decode(fs, encode(fs, word)))
print('decodes correctly:', word == decode(fs, encode(fs, word)))

fs [('b', 1), ('c', 2), ('a', 4)]
dict {'b': '00', 'c': '01', 'a': '1'}
dict {'00': 'b', '01': 'c', '1': 'a'}
encoded: 1111000101
decoded: aaaabcc
decodes correctly: True
